In [1]:
import random_clip_forest
import torch
import clip
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.calibration import CalibratedClassifierCV
import torchvision.transforms.functional as fn
from torchvision.transforms import RandomAffine
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score

In [2]:
clip_model = random_clip_forest.CLIP()

In [3]:
# read in images
df = pd.read_csv('train.csv')
# throw away missing images
df = df.loc[df.image_id.apply(random_clip_forest.img_exists)]
df['Images'] = df['image_id'].apply(random_clip_forest.open_img_id)
# pad all images to a square to not lose information
square_pad = random_clip_forest.SquarePad()

df2 = df.copy()
df2['Images'] = df['Images'].apply(fn.hflip)

df3 = df.copy()
df3['Images'] = [fn.gaussian_blur(img=image,kernel_size=(9,9)) for image in df['Images']]

#df4 = df.copy()
#df4['Images'] = [fn.rotate(img=image,angle=15) for image in df['Images']]

#df4 = df.copy()
#df4['Images'] = [RandomAffine((30,70))(img=image) for image in df['Images']]

#df = pd.concat([df, df2, df3, df4])
df = pd.concat([df, df2, df3])

labelsdf = pd.read_csv('labels.csv')
labels = labelsdf['object'].values.tolist()

img286.jpg
img172.jpg
img244.jpg
img287.jpg


In [4]:
X_train = clip_model.img_embeddings(df['Images'])
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

y_train = np.array([
    random_clip_forest.onehot(lbl) for lbl in df['labels']
]).astype(int)

In [5]:
xgboost_model = XGBClassifier(scale_pos_weight=90, max_delta_step=7,reg_lambda=5)
final = MultiOutputClassifier(xgboost_model)
final.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              gpu_id=None, grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=7, max_depth=None,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))

In [6]:
prediction_on_train = final.predict(X_train)
print(f'f1 score: {f1_score(y_train, prediction_on_train, average="macro"):.3f}')
print(final.score(X_train, y_train))

f1 score: 1.000
1.0


In [7]:
test = np.load('clip_embeddings_test.npz', allow_pickle=True)
X_test = test['embeddings']
X_test = sc.fit_transform(X_test)
test_img_ids = test['img_id']

testdf = pd.read_csv('test.csv')

In [8]:
labelsdf = pd.read_csv('labels.csv')
testlabels = []

for img_id in testdf['image_id']:
    if img_id in test_img_ids:
        prediction = final.predict(X_test[img_id == test_img_ids].reshape(1,-1))
        predicted_labels = random_clip_forest.labelstring(prediction.astype(bool))
        if len(predicted_labels) == 0:
            testlabels.append('l1')
        else:
            testlabels.append(predicted_labels)
        print('='*40)
        print(img_id)
        print(labelsdf.loc[labelsdf.label_id.isin(testlabels[-1].split(' ')), 'object'])
        print('='*40)
    else:
        # default label for the missing images in our test set
        print('test image', img_id, 'missing from images')
        testlabels.append('l0')


testdf['labels'] = testlabels
testdf.to_csv('kea_submissions/xgboost_flipblurreg5.csv', index=False)

img102.jpg
0                people
1                 trees
5             buildings
26        estonian flag
39                flags
41    drone photography
Name: object, dtype: object
img103.jpg
0         people
1          trees
2          grass
15    lamp posts
Name: object, dtype: object
img11.jpg
0    people
1     trees
4      snow
Name: object, dtype: object
img113.jpg
10    nature
Name: object, dtype: object
img114.jpg
44    glasses
51     plants
Name: object, dtype: object
img121.jpg
0     people
1      trees
6      water
19    clouds
22      sand
Name: object, dtype: object
img126.jpg
0                         people
1                          trees
6                          water
38                         barge
40    emajõe-peipsi barge jõmmu
Name: object, dtype: object
img131.jpg
1    trees
Name: object, dtype: object
img136.jpg
44    glasses
Name: object, dtype: object
img137.jpg
1          trees
4           snow
21    stone road
Name: object, dtype: object
img139.jpg
1    